# Read Me — How to Run This Notebook

Follow these steps before running:
- Use kernel: Python 3.10 (aai520-310)
- Create .env at the project root with OPENAI_API_KEY, FRED_API_KEY, and optional NEWSAPI_KEY/NEWS_API_KEY
- Optional: set FORCE_DEMO=1 to skip LLM calls and run offline-friendly demo
- Run cells top-to-bottom. The final report prints and is saved to memory_log.txt
- Optional: set EXPORT_HTML=1 to export this executed notebook to HTML

# Final Submission — CrewAI Orchestrated Run

This clean notebook runs the multi-agent financial analysis using the project's orchestrator and agents.
- Loads environment from .env at project root
- Validates keys and supports NEWSAPI_KEY/NEWS_API_KEY
- Runs full CrewAI pipeline via `src/orchestrator.py`
- Falls back to a quick demo snapshot if needed (quota/offline)
- Optional: export to HTML at the end

In [7]:
# Environment setup and imports
import os, sys, subprocess
from pathlib import Path
from dotenv import load_dotenv

# Resolve project root to current working directory of this notebook
PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"

# Ensure src is importable
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print("Project root:", PROJECT_ROOT)
print("Using src path:", SRC_DIR)

# Load .env explicitly from project root
load_dotenv(PROJECT_ROOT / ".env")

# Optional: ensure requirements are present (no-op if already installed)
REQS = PROJECT_ROOT / "requirements.txt"
if REQS.exists() and os.getenv("INSTALL_REQUIREMENTS", "0") == "1":
    print("Installing requirements (optional)...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQS)], check=False)

Project root: /Users/atulaneja/Documents/GitHub_Class/aai-520-final-project
Using src path: /Users/atulaneja/Documents/GitHub_Class/aai-520-final-project/src


## Agent Modules — Sanity Check

This cell verifies that all CrewAI agent modules import correctly and exposes the expected agent instances.

In [8]:
# Agent Modules — Sanity Check
from agents.critic_agent import critic_agent
from agents.earnings_analyst_agent import earnings_analyst
from agents.market_analyst_agent import market_analyst
from agents.news_analyst_agent import news_analyst
from agents.investment_agents import investment_advisor

agents = [
    ("critic_agent", critic_agent),
    ("earnings_analyst", earnings_analyst),
    ("market_analyst", market_analyst),
    ("news_analyst", news_analyst),
    ("investment_advisor", investment_advisor),
]
for name, ag in agents:
    try:
        tools = getattr(ag, "tools", []) or []
        print(f"✓ {name}: role='{ag.role}', tools={len(tools)}")
    except Exception as e:
        print(f"✗ {name}: import/attr error -> {e}")

✓ critic_agent: role='Quality Assurance Critic', tools=0
✓ earnings_analyst: role='Earnings Analyst', tools=3
✓ market_analyst: role='Macroeconomic Analyst', tools=1
✓ news_analyst: role='News Analyst', tools=1
✓ investment_advisor: role='Investment Advisor', tools=0


## Tasks — Sanity Check


In [9]:
# Tasks — Sanity Check
# Import orchestrator to ensure tasks are wired (agents and contexts assigned)
try:
    from orchestrator import run_analysis as _wired  # noqa: F401
except Exception:
    # Wiring will still be safe-checked below
    pass

from tasks.financial_tasks import (
    memory_retrieval_task,
    earnings_analysis_task,
    news_analysis_task,
    market_analysis_task,
    advisory_draft_task,
    report_critique_task,
)

all_tasks = [
    ("memory_retrieval_task", memory_retrieval_task),
    ("earnings_analysis_task", earnings_analysis_task),
    ("news_analysis_task", news_analysis_task),
    ("market_analysis_task", market_analysis_task),
    ("advisory_draft_task", advisory_draft_task),
    ("report_critique_task", report_critique_task),
]

for name, t in all_tasks:
    try:
        desc = (t.description or "").strip().split("\n")[0]
        ctx_raw = getattr(t, "context", None)
        ctx_len = len(ctx_raw) if isinstance(ctx_raw, (list, tuple)) else 0
        ag = getattr(t, "agent", None)
        ag_role = getattr(ag, "role", None) if hasattr(ag, "role") else None
        print(f"✓ {name}: desc='{desc[:80]}', context={ctx_len}, agent={ag_role}")
    except Exception:
        # Suppress noisy errors (e.g., NotSpecified); skip printing
        continue

✓ memory_retrieval_task: desc='Retrieve any past analysis for the stock MSFT from the memory log using the Read', context=0, agent=Earnings Analyst
✓ earnings_analysis_task: desc='Analyze the most recent annual financial statements for the stock MSFT. First, c', context=1, agent=Earnings Analyst
✓ news_analysis_task: desc='Conduct a comprehensive news analysis for the company associated with the ticker', context=0, agent=News Analyst
✓ market_analysis_task: desc='Analyze the current macroeconomic environment. Fetch the latest data for Gross D', context=0, agent=Macroeconomic Analyst
✓ advisory_draft_task: desc='Synthesize the financial statement analysis, news sentiment analysis, and macroe', context=3, agent=Investment Advisor
✓ report_critique_task: desc='Review the DRAFT investment report provided in the context. Your role is to act ', context=1, agent=Quality Assurance Critic


##  Utility: Demo fallback when full Crew run is unavailable

In [10]:
import requests
from datetime import datetime

def demo_fallback_snapshot(ticker: str) -> str:
    print("[Demo mode] Running simplified snapshot.\n")
    # Stock via yfinance
    price = None
    name = ticker
    sector = ""
    try:
        import yfinance as yf
        stock = yf.Ticker(ticker)
        info = getattr(stock, "info", {}) or {}
        name = info.get("longName") or name
        sector = info.get("sector") or sector
        price = info.get("regularMarketPrice")
        if price is None:
            hist = stock.history(period="5d")
            if not hist.empty:
                price = float(hist["Close"].iloc[-1])
    except Exception:
        pass

    # News via NewsAPI
    news_titles = []
    key = os.getenv("NEWSAPI_KEY") or os.getenv("NEWS_API_KEY")
    if key:
        try:
            r = requests.get(
                "https://newsapi.org/v2/everything",
                params={"q": ticker, "sortBy": "publishedAt", "pageSize": 5, "language": "en", "apiKey": key},
                timeout=20,
            )
            data = r.json() if r.ok else {}
            for a in (data.get("articles") or [])[:5]:
                title = a.get("title") or ""
                if title:
                    news_titles.append(title)
        except Exception:
            pass

    # Macro via FRED
    fred_summary = {}
    fred_key = os.getenv("FRED_API_KEY")
    if fred_key:
        try:
            from fredapi import Fred
            fred = Fred(api_key=fred_key)
            cpi = fred.get_series("CPIAUCSL").tail(1)
            gdp = fred.get_series("GDP").tail(1)
            if not cpi.empty:
                fred_summary["CPIAUCSL"] = float(cpi.iloc[0])
            if not gdp.empty:
                fred_summary["GDP"] = float(gdp.iloc[0])
        except Exception:
            pass

    lines = [
        f"Demo Investment Snapshot ({datetime.now().strftime('%Y-%m-%d %H:%M')})",
        f"Ticker: {ticker}",
        f"Company: {name}",
        f"Sector: {sector}" if sector else "Sector: n/a",
        f"Last Price: {price}" if price is not None else "Last Price: n/a",
    ]
    if news_titles:
        lines.append("Top News: " + " | ".join(news_titles[:3]))
    if fred_summary:
        lines.append("Macro: " + ", ".join(f"{k}={v}" for k, v in fred_summary.items()))

    report = "\n".join(lines)
    print(report)
    return report

## Validate environment and run orchestration

In [11]:
import traceback

# Accept NEWS_API_KEY alias
if "NEWSAPI_KEY" not in os.environ and os.getenv("NEWS_API_KEY"):
    os.environ["NEWSAPI_KEY"] = os.getenv("NEWS_API_KEY")

required_keys = ["OPENAI_API_KEY", "FRED_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    print("Missing required keys:", missing)
    print("Add them to .env and rerun. Falling back to demo if FORCE_DEMO=1 or OPENAI key missing.")

use_demo = os.getenv("FORCE_DEMO", "0") == "1" or ("OPENAI_API_KEY" in missing)

ticker = os.getenv("DEMO_TICKER", "MSFT")

if use_demo:
    _ = demo_fallback_snapshot(ticker)
else:
    try:
        # Orchestrated CrewAI run
        from orchestrator import run_analysis
        print(f"Starting multi-agent analysis for {ticker}...")
        run_analysis(ticker)
    except Exception as e:
        print("[ERROR] Orchestrator run failed:", e)
        print("Falling back to demo snapshot...\n")
        try:
            _ = demo_fallback_snapshot(ticker)
        except Exception as e2:
            print("[ERROR] Demo fallback also failed:", e2)
            traceback.print_exc()

Starting multi-agent analysis for MSFT...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3ca5bd3a-a072-4e6f-8656-02f108eb4949                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Task: Retrieve any past analysis for the stock MSFT from the memory log using the Read Memory Tool.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Thought: I need to check if there is any past analysis for the stock MSFT in the memory log.                   │
│                                                                                                                 │
│  Using Tool: Read Memory Tool                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Critic                                                                                │
│                                                                                                                 │
│  Task: Review the DRAFT investment report provided in the context. Your role is to act as a skeptical quality   │
│  assurance analyst. Critique the report based on the following principles:                                      │
│  1. Clarity and Cohesion: Is the main investment thesis clear and easy to understand?                           │
│  2. Evidentiary Support: Are all claims backed by specific data points from the analyses?                       │
│  3. Risk Assessment: Does the report adequately identify and discuss the primary risks?                         │
│                                                                                                                 │
│  After your critique, produce a FINAL, polished investment report that incorporates your feedback and provides  │
│  a definitive recommendation for MSFT.                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Critic                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Polished Investment Recommendation Report for Microsoft Corporation (MSFT)**                                 │
│                                                                                                                 │
│  **Recommendation: Buy**                                                                                        │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│                                                                                                                 │
│  1. **Financial Performance Analysis:**                                                                         │
│     - Microsoft Corporation has delivered an exceptional financial performance, with total revenues reaching    │
│  **$281.72 billion** and a net income of **$101.83 billion**, resulting in a **diluted EPS of $13.64**. This    │
│  robust performance highlights the company's strong demand across its segments, particularly in cloud           │
│  computing, productivity software, and gaming. Such high revenue generation reinforces confidence in the        │
│  sustainability of its business model.                                                                          │
│     - The company's **gross profit margin of 68.8%** not only indicates an ability to maintain high             │
│  profitability levels but also reflects significant pricing power and operational efficiency. Coupled with an   │
│  operating income of **$128.53 billion**, these figures underscore Microsoft’s effective cost management        │
│  strategies and strategic focus on profitability.                                                               │
│     - An analysis of the balance sheet reveals a strong financial foundation, with total assets of **$619.00    │
│  billion** and total liabilities of **$275.52 billion**, yielding equity of **$343.48 billion**. The            │
│  debt-to-equity ratio of approximately **0.04** suggests a conservative leverage strategy, minimizing           │
│  financial risk. With net debt at only **$12.91 billion**, Microsoft demonstrates a low reliance on debt,       │
│  which further reduces financial vulnerability.                                                                 │
│     - The operating cash flow is robust at **$136.16 billion**, and a free cash flow of **$71.61 billion**      │
│  indicates substantial financial flexibility. This availability of cash enables Microsoft to pursue future      │
│  growth investments and return capital to shareholders efficiently.                                             │
│                                                                                                                 │
│  2. **News Sentiment Analysis:**                                                                                │
│     - While specific recent news sentiment data is not available, historical trends indicate that Microsoft     │
│  retains a positive reputation in the investment community, especially concerning its innovations in            │
│  artificial intelligence (AI) and cloud services.                                                               │
│     - The continued development and expansion of the Az

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6489642f-5067-483b-889f-4cda50586e73                                                                     │
│  Agent: Quality Assurance Critic                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3ca5bd3a-a072-4e6f-8656-02f108eb4949                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Polished Investment Recommendation Report for Microsoft Corporation (MSFT)**                   │
│                                                                                                                 │
│  **Recommendation: Buy**                                                                                        │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│                                                                                                                 │
│  1. **Financial Performance Analysis:**                                                                         │
│     - Microsoft Corporation has delivered an exceptional financial performance, with total revenues reaching    │
│  **$281.72 billion** and a net income of **$101.83 billion**, resulting in a **diluted EPS of $13.64**. This    │
│  robust performance highlights the company's strong demand across its segments, particularly in cloud           │
│  computing, productivity software, and gaming. Such high revenue generation reinforces confidence in the        │
│  sustainability of its business model.                                                                          │
│     - The company's **gross profit margin of 68.8%** not only indicates an ability to maintain high             │
│  profitability levels but also reflects significant pricing power and operational efficiency. Coupled with an   │
│  operating income of **$128.53 billion**, these figures underscore Microsoft’s effective cost management        │
│  strategies and strategic focus on profitability.                                                               │
│     - An analysis of the balance sheet reveals a strong financial foundation, with total assets of **$619.00    │
│  billion** and total liabilities of **$275.52 billion**, yielding equity of **$343.48 billion**. The            │
│  debt-to-equity ratio of approximately **0.04** suggests a conservative leverage strategy, minimizing           │
│  financial risk. With net debt at only **$12.91 billion**, Microsoft demonstrates a low reliance on debt,       │
│  which further reduces financial vulnerability.                                                                 │
│     - The operating cash flow is robust at **$136.16 billion**, and a free cash flow of **$71.61 billion**      │
│  indicates substantial financial flexibility. This availability of cash enables Microsoft to pursue future      │
│  growth investments and return capital to shareholders efficiently.                                             │
│                                                                                                                 │
│  2. **News Sentiment Analysis:**                                                                                │
│     - While specific recent news sentiment data is not available, historical trends indicate that Microsoft     │
│  retains a positive reputation in the investment community, especially concerning its innovations in            │
│  artificial intelligence (AI) and cloud services.     



########################
## Final Analysis Report:
########################
**Polished Investment Recommendation Report for Microsoft Corporation (MSFT)**

**Recommendation: Buy**

**Rationale:**

1. **Financial Performance Analysis:**
   - Microsoft Corporation has delivered an exceptional financial performance, with total revenues reaching **$281.72 billion** and a net income of **$101.83 billion**, resulting in a **diluted EPS of $13.64**. This robust performance highlights the company's strong demand across its segments, particularly in cloud computing, productivity software, and gaming. Such high revenue generation reinforces confidence in the sustainability of its business model.
   - The company's **gross profit margin of 68.8%** not only indicates an ability to maintain high profitability levels but also reflects significant pricing power and operational efficiency. Coupled with an operating income of **$128.53 billion**, these figures underscore Microsoft’s effective cost ma

## Optional: HTML export of this executed notebook

In [12]:
# Set EXPORT_HTML=1 in environment to enable.
import os
if os.getenv("EXPORT_HTML", "0") == "1":
    try:
        import nbformat, nbconvert
        from nbconvert import HTMLExporter
        from nbconvert.writers import FilesWriter
        import json
        from pathlib import Path
        
        nb_path = Path(__file__).resolve() if '__file__' in globals() else None
        if nb_path is None:
            # When running inside Jupyter UI, provide a default path
            nb_path = Path.cwd() / "Final_Submission_CrewAI_Run.ipynb"
        
        # Best-effort read and export
        with open(nb_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        html_exporter = HTMLExporter()
        (body, resources) = html_exporter.from_notebook_node(nb)
        out_file = nb_path.with_suffix('.html')
        with open(out_file, 'w', encoding='utf-8') as f:
            f.write(body)
        print("Exported HTML:", out_file)
    except Exception as e:
        print("[WARN] HTML export skipped:", e)